<a href="https://colab.research.google.com/github/Linamay-hub/dashboard-proyecto/blob/main/eleccioncarrera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler

# Fijamos una semilla para que los resultados aleatorios puedan reproducirse
np.random.seed(42)

# Cantidad de estudiantes simulados
n = 120

# Generamos aleatoriamente las variables base
genero = np.random.choice([0, 1], size=n)  # 0 = masculino, 1 = femenino
calificaciones = np.random.normal(loc=80, scale=10, size=n).clip(60, 100).astype(int)  # promedio entre 60 y 100
interes_ciencias = np.random.randint(0, 11, size=n)       # interés de 0 a 10
interes_humanidades = np.random.randint(0, 11, size=n)
interes_artes = np.random.randint(0, 11, size=n)

# Creamos un DataFrame con los datos simulados
df = pd.DataFrame({
    'Genero': genero,
    'Calificaciones': calificaciones,
    'Ciencias': interes_ciencias,
    'Humanidades': interes_humanidades,
    'Artes': interes_artes
})

# Calculamos probabilidades de elección de carrera, basadas en el interés relativo de cada estudiante
probs = df[['Ciencias', 'Humanidades', 'Artes']].astype(float)
probs_sum = probs.sum(axis=1).replace(0, 1e-6)  # evitamos división por cero
normalized_probs = probs.div(probs_sum, axis=0)  # normalizamos para que sumen 1

# Asignamos la elección de carrera de forma aleatoria pero influenciada por el interés
df['Eleccion'] = [
    np.random.choice([1, 2, 3], p=normalized_probs.iloc[i].values)
    for i in range(len(df))
]
# Elección: 1 = Ciencias, 2 = Humanidades, 3 = Artes

# Variables independientes para el modelo
X = df[['Genero', 'Calificaciones', 'Ciencias', 'Humanidades', 'Artes']]
y = df['Eleccion']  # variable dependiente (respuesta)

# Escalamos las variables para mejorar la estabilidad del modelo
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Añadimos una constante (necesaria para el modelo estadístico)
X_scaled = sm.add_constant(X_scaled)

# Creamos el modelo Logit Multinomial usando statsmodels
modelo = sm.MNLogit(y, X_scaled)

# Ajustamos el modelo a los datos
resultado = modelo.fit()

# Mostramos el resumen del modelo (coeficientes, significancia, etc.)
print(resultado.summary())

Optimization terminated successfully.
         Current function value: 0.896689
         Iterations 6
                          MNLogit Regression Results                          
Dep. Variable:               Eleccion   No. Observations:                  120
Model:                        MNLogit   Df Residuals:                      108
Method:                           MLE   Df Model:                           10
Date:                Thu, 29 May 2025   Pseudo R-squ.:                  0.1805
Time:                        02:00:28   Log-Likelihood:                -107.60
converged:                       True   LL-Null:                       -131.30
Covariance Type:            nonrobust   LLR p-value:                 7.996e-07
Eleccion=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1636      0.972     -0.168      0.866      -2.068       1.741
x1            -0.4792      0.

In [12]:
import pandas as pd
from scipy.stats import ttest_ind

# Datos (resumen de tus 35 registros en un DataFrame)
datos = {
    'Genero': [1,0,1,1,0,0,1,1,0,0,1,1,0,0,1,1,0,1,0,1,1,0,0,1,0,1,1,0,1,1,0,0,1,0,1],
    'Calificaciones': [79,88,72,93,68,85,76,90,71,79,84,73,89,67,91,85,78,92,65,88,70,95,75,82,60,70,74,77,83,86,69,61,80,63,87],
    'Ciencias': [6,7,5,8,4,6,5,7,5,6,7,5,8,4,7,8,6,9,5,7,4,8,5,7,4,4,6,5,7,8,3,2,5,3,7],
    'Humanidades': [5,3,6,2,7,4,6,3,5,4,3,6,2,7,3,4,7,3,8,4,6,3,7,5,8,6,4,7,3,4,8,9,6,9,4],
    'Artes': [1,4,0,5,0,3,0,4,1,2,3,1,4,0,5,2,1,4,0,3,0,5,2,1,0,2,1,2,4,4,2,1,3,1,5],
    'Eleccion': [2,1,2,1,2,1,2,1,3,3,3,2,1,2,1,1,2,1,2,1,2,1,2,3,3,2,2,2,1,1,2,3,3,3,1]
}

df = pd.DataFrame(datos)

# Filtramos calificaciones por elección
humanidades = df[df['Eleccion'] == 2]['Calificaciones']
artes = df[df['Eleccion'] == 3]['Calificaciones']

# Prueba t
t_stat, p_value = ttest_ind(humanidades, artes, equal_var=False)

# Resultados
print(f"Promedio Humanidades: {humanidades.mean():.2f}")
print(f"Promedio Artes: {artes.mean():.2f}")
print(f"Valor t: {t_stat:.3f}")
print(f"Valor p: {p_value:.4f}")

# Interpretación
if p_value < 0.05:
    print("✅ Hay una diferencia significativa entre las calificaciones de Humanidades y Artes.")
else:
    print("❌ No hay una diferencia significativa entre las calificaciones de Humanidades y Artes.")

Promedio Humanidades: 72.36
Promedio Artes: 72.50
Valor t: -0.038
Valor p: 0.9703
❌ No hay una diferencia significativa entre las calificaciones de Humanidades y Artes.
